In [ ]:
using Plots
using Distributions
using Distances
using DataFrames
using CSV
using FFTW
using CairoMakie
using PairPlots

In [ ]:
# FUNCTION FOR THE PLOTTING OF THE POSTERIORS AS A CORNER PLOT

# function to load the saved solution of a SDDE
function load_solution(filename::String = "simulated_sol.csv")
  df = CSV.read(filename, DataFrame)

  t = Vector(df.Time)
  u = Vector(df.u)
  du = Vector(df.du)

  return t, u, du
end

# function to load the result object of a sABC algorithm
function load_result()
  curr_path = pwd()
  
  filenames = ["eps_hist.csv", "u_hist.csv", "rho_hist.csv"]
  variables = []
  for filename in filenames
      path = joinpath(curr_path, filename)
      push!(variables, CSV.read(path, DataFrame))
      println("Data loaded from file: $path")
  end
  eps_hist, u_hist, rho_hist = variables

  filename = "population.csv"
  path = joinpath(curr_path, filename)
  posterior_params = CSV.read(path, DataFrame)
  println("Posterior parameters loaded from file: $path")
  
  return eps_hist, u_hist, rho_hist, posterior_params
end

function post_plotting(post_par, true_vals)
  pairplot(
    posterior_params,
    PairPlots.Truth(
        (;
            N_value = true_vals[1],
            T_value = true_vals[2],
            tau_value = true_vals[3],
            sigma_value = true_vals[4],
            Bmax_value = true_vals[5]
        ),
        
        label="True Values"
    )
  )
end

In [ ]:
function plot_fourier_spectrum(t::Vector{Float64}, u::Vector{Float64})
  Fs = 1 / (t[2] - t[1])
  N = length(u)
  freqs = fftfreq(N, Fs)

  Plots.plot(freqs, abs.(fft(u)), xlabel="Frequency (Hz)", ylabel="Magnitude", title="Magnitude Spectrum", legend=false, xlims =(0, 1))
end

In [ ]:
plot_fourier_spectrum(t, u)

In [ ]:
t, u, du = load_solution()

In [ ]:
eps_hist, u_hist, rho_hist, posterior_params = load_result()

In [ ]:
p1 = Plots.plot(vec(Matrix(eps_hist)), title="Epsilon History", xlabel="Iteration", legend=false, yscale=:log10)
p2 = Plots.plot(vec(Matrix(u_hist)), title="U History", xlabel="Iteration", legend=false)

# Combine plots
combined_plot = Plots.plot(p1, p2, layout=(1, 2), size=(1000, 400))

display(combined_plot)

In [ ]:
rho_plots = []

for i in 1:20
  push!(rho_plots, Plots.plot(Matrix(rho_hist)[i, :], title="Rho History Stat $i", xlabel="Iteration", legend=false, yscale=:log10))
end

combined_plot = Plots.plot(rho_plots..., layout=(5, 4), size=(1200, 1800))

display(combined_plot)

In [ ]:
labels = ["rho$i" for i in 1:size(Matrix(rho_hist), 1)]

all_rho_plot = plot(title="All Rho History", xlabel="Iteration", ylabel="Rho", legend=true, yscale=:log10)
for i in 1:size(rho_singeps, 1)
  plot!(1:size(Matrix(rho_hist), 2), Matrix(rho_hist)[i, :], label=labels[i])
end

# Display the plot
display(all_rho_plot)

In [ ]:
post_plotting(posterior_params, [6.2, 3.1, 3.5, 0.04, 6.0])